In [2]:
!pip install duckdb

  Using cached duckdb-0.5.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)


In [1]:
import duckdb

In [2]:
# basic SQL API

# connect to an in-memory temporary database
conn = duckdb.connect()

# if you want, you can create a cursor() like described in PEP 249 but its fully redundant
cursor = conn.cursor()

In [3]:
conn.execute("CREATE TABLE test_table (i INTEGER, j STRING)")

In [4]:
# add some data
conn.execute("INSERT INTO test_table VALUES (1, 'one')")

In [5]:
# we can use placeholders for parameters
conn.execute("INSERT INTO test_table VALUES (?, ?)", [2, 'two'])

# we can provide multiple sets of parameters to executemany()
conn.executemany("INSERT INTO test_table VALUES (?, ?)", [[3, 'three'], [4, 'four']])

# fetch as pandas data frame
print(conn.execute("SELECT * FROM test_table").fetchdf())

# fetch as list of masked numpy arrays, cleaner when handling NULLs
print(conn.execute("SELECT * FROM test_table").fetchnumpy())

   i      j
0  1    one
1  2    two
2  3  three
3  4   four
{'i': array([1, 2, 3, 4], dtype=int32), 'j': array(['one', 'two', 'three', 'four'], dtype=object)}


In [6]:
import pandas as pd

In [7]:
test_df = pd.DataFrame.from_dict({"i":[1, 2, 3, 4], "j":["one", "two", "three", "four"]})

# make this data frame available as a view in duckdb
conn.register("test_df", test_df)
print(conn.execute("SELECT j FROM test_df WHERE i > 1").fetchdf())

       j
0    two
1  three
2   four


In [8]:
# create a "relation" from a pandas data frame with an existing connection
rel = conn.from_df(test_df)
print(rel)

# alternative shorthand, use a built-in default connection to create a relation from a pandas data frame
rel = duckdb.df(test_df)
print(rel)

---------------------
--- Relation Tree ---
---------------------
pandas_scan(140223100473008)

---------------------
-- Result Columns  --
---------------------
- i (BIGINT)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
BIGINT	VARCHAR	
[ Rows: 4]
1	one
2	two
3	three
4	four



---------------------
--- Relation Tree ---
---------------------
pandas_scan(140223862064656)

---------------------
-- Result Columns  --
---------------------
- i (BIGINT)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
BIGINT	VARCHAR	
[ Rows: 4]
1	one
2	two
3	three
4	four





In [17]:
# create a relation from a CSV file

# first create a CSV file from our pandas example 
import tempfile, os
temp_file_name = os.path.join(tempfile.mkdtemp(), next(tempfile._get_candidate_names()))
test_df.to_csv(temp_file_name, index=False)

# now create a relation from it
rel = duckdb.from_csv_auto(temp_file_name)
print(rel)

---------------------
--- Relation Tree ---
---------------------
read_csv_auto(/tmp/tmpovrvu0j7/a7bgwcaw)

---------------------
-- Result Columns  --
---------------------
- i (INTEGER)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
INTEGER	VARCHAR	
[ Rows: 4]
1	one
2	two
3	three
4	four





In [18]:
# create a relation from an existing table
rel = conn.table("test_table")
print(rel)

---------------------
--- Relation Tree ---
---------------------
Scan Table [test_table]

---------------------
-- Result Columns  --
---------------------
- i (INTEGER)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
INTEGER	VARCHAR	
[ Rows: 4]
1	one
2	two
3	three
4	four





In [19]:
# a relation has an alias (like a table name)
print(rel.alias)

# we can change the alias, useful for (self)joins for example
rel2 = rel.set_alias('new_alias')
print(rel2.alias)

# we can inspect the type of a relation
print(rel.type)

test_table
new_alias
TABLE_RELATION


In [20]:
# or the column names that are in it
print(rel.columns)

# or the types of those columns
print(rel.types)

# now we can apply some operators to the relation
# filter the relation
print(rel.filter('i > 1'))

# project the relation, get some columns
print(rel.project('i, j'))

['i', 'j']
['INTEGER', 'VARCHAR']
---------------------
--- Relation Tree ---
---------------------
Filter [(i) > (1)]
  Scan Table [test_table]

---------------------
-- Result Columns  --
---------------------
- i (INTEGER)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
INTEGER	VARCHAR	
[ Rows: 3]
2	two
3	three
4	four



---------------------
--- Relation Tree ---
---------------------
Projection [i as , j as ]
  Scan Table [test_table]

---------------------
-- Result Columns  --
---------------------
- i (INTEGER)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
INTEGER	VARCHAR	
[ Rows: 4]
1	one
2	two
3	three
4	four





In [24]:
# or transform them
print(rel.project('i + 1'))

# order the relation
print(rel.order('j'))

# limit the rows returned
print(rel.limit(2))

# skip the first row and limit the number of results
print(rel.limit(2, offset=1))

# of course these things can be chained
print(rel.filter('i > 1').order('j').project('i + 1').limit(2))

# aggregate the relation
print(rel.aggregate("sum(i)"))

# non-aggregated columns create implicit grouping
print(rel.aggregate("j, sum(i)"))

# we can also explicit group the relation before aggregating
print(rel.aggregate("sum(i)", "j"))

# distinct values
print(rel.distinct())

---------------------
--- Relation Tree ---
---------------------
Projection [(i) + (1) as ]
  Scan Table [test_table]

---------------------
-- Result Columns  --
---------------------
- (i) + (1) (INTEGER)

---------------------
-- Result Preview  --
---------------------
(i) + (1)	
INTEGER	
[ Rows: 4]
2
3
4
5



---------------------
--- Relation Tree ---
---------------------
Order [j DESC]
  Scan Table [test_table]

---------------------
-- Result Columns  --
---------------------
- i (INTEGER)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
INTEGER	VARCHAR	
[ Rows: 4]
4	four
1	one
3	three
2	two



---------------------
--- Relation Tree ---
---------------------
Limit 2
  Scan Table [test_table]

---------------------
-- Result Columns  --
---------------------
- i (INTEGER)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
INTEGER	VARCHAR	
[ Rows: 2]
1	one
2	two



---------------------
--- Relation Tree -